# 잠재요인 협업 필터링

잠재 요인 협업 필터링
- 사용자-아이템 평점 행렬 속에 숨어 있는 잠재 요인을 추출해서
- 추천 예측을 수행하는 기법
- 대규모 다차원 행렬을 분해하는 과정에서 잠재 요인을 추출하고
- 잠재 요인 기반으로 사용자-아이템 평점 행렬을 재 구성하면서 추천 구현
- 넷플릭스 경연 대회에서 사용되면서 유명해짐

- 사용자-아이템 평점 행렬 데이터만을 이용해 '잠재 요인'을 끄집어 내는 것을 의미하는데
- 사실 잠재 요인을 무엇이다라고 특징짓기는 어려움
- 그러나 잠재 요인을 기반으로 다차원 희소 행렬인 사용자-아이템 행렬 데이터를
- 저차원 밀집 행렬의 사용자-잠재 요인 행렬과 
- 아이템-잠재 요인 행렬의 전치 행렬(잠재요인-아이템 행렬)로 분해할 수 있고
- 이렇게 분해된 두 행렬의 내적 곱으로 결합하면서 새로운 예측 사용자-아이템 평점 행렬 데이터를 만들어
- 사용자가 아직 평점을 부여하지 않은 아이템에 대한 예측 평점을 생성하는 것이
- 잠재 요인 협력 필터링 알고리즘의 핵심

- 행렬 분해에 의해 추출되는 잠재 요인이 정확히 어떤 것인지는 알 수 없지만
- 영화 평점 기반의 사용자-아이템 평점 행렬 데이터인 경우
- 영화가 가지는 장르별 선호도를 잠재 요인으로 가정할 수 있음

- 즉, 사용자-잠재 요인 행렬 : 사용자의 영화 장르에 대한 선호도를 잠재 요인으로 정의
- 아이템-잠재 요인 행렬 : 영화의 장르별 특성 값을 잠재 요인으로 정의

평점이란 
- 사용자의 특정 영화 장르에 대한 선호도와 
- 개별 영화의 그 장르적 특성값을 반영해서 결정된다고 생각할 수 있음
- 예로, 사용자가 액션 영화를 매우 좋아하고, 
- 특정 영화가 액션 영화의 특성이 매우 크다면
- 사용자가 해당 영화에 높은 평점을 줄 것임
- 따라서 평점은 사용자의 장르별 선호도 벡터와
- 영화의 장르별 특성 벡터를 서로 곱해서 만들 수 있음

### 행렬 분해를 이용한 잠재 요인 협업 필터링 예제

In [5]:
# 행렬 분해하는 함수

def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda = 0.01):
    num_users, num_items = R.shape
    # P와 Q 매트릭스의 크기를 지정하고 정규분포를 가진 랜덤한 값으로 입력합니다. 
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size=(num_users, K))
    Q = np.random.normal(scale=1./K, size=(num_items, K))

    break_count = 0
       
    # R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장. 
    non_zeros = [ (i, j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0 ]

In [8]:
# SGD기법으로 P와 Q 매트릭스를 계속 업데이트. 
for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
            # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij * Q[j, :] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij * P[i, :] - r_lambda*Q[j,:])
       
    rmse = get_rmse(R, P, Q, non_zeros)
    if (step % 10) == 0 :
        print("### iteration step : ", step," rmse : ", rmse)
            
return P, Q

NameError: name 'steps' is not defined

In [10]:
# 데이터 로드 및 사용자-아이템 평점 행렬 생성

import pandas as pd
import numpy as np

movies = pd.read_csv('./data/movielens/movies.csv')
ratings = pd.read_csv('./data/movielens/ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')

# title 컬럼을 얻기 이해 movies 와 조인 수행
rating_movies = pd.merge(ratings, movies, on='movieId')

# columns='title' 로 title 컬럼으로 pivot 수행. 
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')